O dataset escolhido para o trabalho foi o arquivo Weka-3-8\data\breast-cancer.arff, contido na instalação do Weka. A feature utilziada como label foi justamente aquela chamada de class, que assume dois valores possíveis: recurrence-events e no-recurrence-events.

O scikit learn possui basicamente 3 implementações para Naive Bayes:

* Naive Bayes Gaussiano
* Naive Bayes Multinomial (com a "variação" Complemento )
* Naive Bayes Bernoulli

Tanto o Multinomial, quanto o Bernoulli, são adequados para dados discretos. Bernoulli, entretanto, é projeto para features binárias. 

Na execução no Weka, tivemos o seguinte resultado (utilizando k-fold cross validation):

```
=== Summary ===

Correctly Classified Instances         205               71.6783 %
Incorrectly Classified Instances        81               28.3217 %
Kappa statistic                          0.2857
Mean absolute error                      0.3272
Root mean squared error                  0.4534
Relative absolute error                 78.2086 %
Root relative squared error             99.1872 %
Total Number of Instances              286     

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,836    0,565    0,778      0,836    0,806      0,288    0,701     0,837     no-recurrence-events
                 0,435    0,164    0,529      0,435    0,477      0,288    0,701     0,514     recurrence-events
Weighted Avg.    0,717    0,446    0,704      0,717    0,708      0,288    0,701     0,741     

=== Confusion Matrix ===

   a   b   <-- classified as
 168  33 |   a = no-recurrence-events
  48  37 |   b = recurrence-events
```

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB

In [2]:
data = pd.read_csv('breast.csv')
print(data)

       age menopause tumor-size inv-nodes node-caps  deg-malig breast  \
0    40-49   premeno      15-19       0-2       yes          3  right   
1    50-59      ge40      15-19       0-2        no          1  right   
2    50-59      ge40      35-39       0-2        no          2   left   
3    40-49   premeno      35-39       0-2       yes          3  right   
4    40-49   premeno      30-34       3-5       yes          2   left   
5    50-59   premeno      25-29       3-5        no          2  right   
6    50-59      ge40      40-44       0-2        no          3   left   
7    40-49   premeno      10-14       0-2        no          2   left   
8    40-49   premeno        0-4       0-2        no          2  right   
9    40-49      ge40      40-44     15-17       yes          2  right   
10   50-59   premeno      25-29       0-2        no          2   left   
11   60-69      ge40      15-19       0-2        no          2  right   
12   50-59      ge40      30-34       0-2        no

Os dados são mapeados para valores numéricos, para que funcionem com a implementação em questão. Há linhas com o valor '?', ou seja, desconhecido, em diversas colunas. Optei por remover essas linhas, uma vez que algum valor deve ser informado (a implementação não trabalha com valores None, ou seja, NaN).

In [3]:
age_mapping={'10-19':1,'20-29':2,'30-39':3,'40-49':4,'50-59':5,'60-69':6,'70-79':7,'80-89':8,'90-99':9,'?':None}
menopause_mapping={'lt40':1,'ge40':2,'premeno':3,'?':None}
tumor_size_mapping={'0-4':1,'5-9':2,'10-14':3,'15-19':4,'20-24':5,'25-29':6,'30-34':7,'35-39':8,'40-44':9,'45-49':10,'50-54':11,'55-59':12,'?':None}
inv_nodes_mapping={'0-2':1,'3-5':2,'6-8':3,'9-11':4,'12-14':5,'15-17':6,'18-20':7,'21-23':8,'24-26':9,'27-29':10,'30-32':11,'33-35':12,'36-39':13,'?':None}
node_caps_mapping={'yes':1,'no':0,'?':None}
deg_malig_mapping={'1':1,'2':2,'3':3,'?':None}
breast_mapping={'left':1,'right':2,'?':None}
breast_quad_mapping={'left_up':1,'left_low':2,'right_up':3,'right_low':4,'central':5,'?':None}
irradiat_mapping={'yes':1,'no':0,'?':None}
clazz_mapping={'no-recurrence-events':0,'recurrence-events':1,'?':None}

numerical_data=data.replace({'age':age_mapping,
              'menopause':menopause_mapping,
              'tumor-size':tumor_size_mapping,
              'inv-nodes':inv_nodes_mapping,
              'node-caps':node_caps_mapping,
              'breast':breast_mapping,
              'breast-quad':breast_quad_mapping,
              'irradiat':irradiat_mapping,
              'class':clazz_mapping})

print(numerical_data.shape)

numerical_data=numerical_data.dropna()

print(numerical_data.shape)

(286, 10)
(277, 10)


Separação dos dados em dados de treinamento e dados de teste.

In [4]:
X = numerical_data.iloc[:,:-1]
y = numerical_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [5]:
clf = BernoulliNB()
clf.fit(X_train, y_train.astype('int'))

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

Abaixo a acurácia média do treinamento: aproximadamente 67,8571%. A acurácia do executado pelo Weka foi de 71,6783%. A diferença foi inferior a 5% e pode ter sido causada tanto pelo tratamento dos dados, com exclusão das linhas com valores desconhecidos, como pela própria implementação do algoritmo (ou ambos).

In [6]:
clf.score(X_test, y_test.astype('int'))

0.6785714285714286